In [1]:
pip install ray[air]==2.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.4 MB

In [ ]:
import sys
import time
from collections import Counter

import ray

In [ ]:
@ray.remote
def gethostname(x):
    import platform
    import time

    time.sleep(0.01)
    return x + (platform.node(),)


def wait_for_nodes(expected):
    # Wait for all nodes to join the cluster.
    while True:
        resources = ray.cluster_resources()
        node_keys = [key for key in resources if "node" in key]
        num_nodes = sum(resources[node_key] for node_key in node_keys)
        if num_nodes < expected:
            print(
                "{} nodes have joined so far, waiting for {} more.".format(
                    num_nodes, expected - num_nodes
                )
            )
            sys.stdout.flush()
            time.sleep(1)
        else:
            break


def main():
    wait_for_nodes(2)

    # Check that objects can be transferred from each node to each other node.
    for i in range(10):
        print("Iteration {}".format(i))
        results = [gethostname.remote(gethostname.remote(())) for _ in range(100)]
        print(Counter(ray.get(results)))
        sys.stdout.flush()

    print("Success!")
    sys.stdout.flush()

In [ ]:
ray.init("ray://example-cluster-kuberay-head-svc:10001")

Python version:,3.9.12
Ray version:,2.2.0
Dashboard:,http://10.8.8.11:8265


In [ ]:
main()

Iteration 0
Counter({('example-cluster-kuberay-worker-workergroup-9bnxn', 'example-cluster-kuberay-worker-workergroup-9bnxn'): 38, ('example-cluster-kuberay-worker-workergroup-9bnxn', 'example-cluster-kuberay-head-fd9g6'): 27, ('example-cluster-kuberay-head-fd9g6', 'example-cluster-kuberay-head-fd9g6'): 26, ('example-cluster-kuberay-head-fd9g6', 'example-cluster-kuberay-worker-workergroup-9bnxn'): 9})
Iteration 1
Counter({('example-cluster-kuberay-head-fd9g6', 'example-cluster-kuberay-head-fd9g6'): 31, ('example-cluster-kuberay-worker-workergroup-9bnxn', 'example-cluster-kuberay-worker-workergroup-9bnxn'): 26, ('example-cluster-kuberay-head-fd9g6', 'example-cluster-kuberay-worker-workergroup-9bnxn'): 23, ('example-cluster-kuberay-worker-workergroup-9bnxn', 'example-cluster-kuberay-head-fd9g6'): 20})
Iteration 2
Counter({('example-cluster-kuberay-head-fd9g6', 'example-cluster-kuberay-head-fd9g6'): 33, ('example-cluster-kuberay-worker-workergroup-9bnxn', 'example-cluster-kuberay-worker-w